In [1]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import csv
import stats_arrays
import scipy as sp
import pandas as pd
import pickle
import h5py
import time

In [2]:
bw.projects.set_current('iw_integration')

In [3]:
DB_eiv33=bw.Database('ecoinvent 3.3 cutoff')
biosphere3=bw.Database('biosphere3')

In [42]:
#######Functions to calculate correlated LCI######

def calculate_ratio_corr_lognormal(gmean_Xi,gmean_Yj,gsigma_Xi,gsigma_Yj):
    
    #We assume that Xi and Yj are perfeclty correlated
    gsigma_XiYj=gsigma_Xi*gsigma_Yj
    
    gmean_Aij=gmean_Xi-gmean_Yj
    gsigma_Aij=np.sqrt((gsigma_Xi**2)+(gsigma_Yj**2)-(2*gsigma_XiYj))
    
    if np.isnan(gsigma_Aij) or gsigma_Aij==0:
        gsigma_Aij=0.0000000000001
    
    return {'loc':gmean_Aij,'scale':gsigma_Aij,'uncertainty_type':2};


#Only once for the DB
def calculate_ratio_corr_lognormal_for_DB(corr_ef_code_input,
                                          corr_ef_code_output,
                                          corr_twin_ef_input_output,
                                          biosphere_dict,
                                          bio_params,biosphere_names):
    
    #corr_bio_params=bio_params
    ratio_rv_dict_per_activity={}
    inv_biosphere_dict={indice: ef for ef, indice in biosphere_dict.items()}
    
    set_efs=set([key[1] for key in biosphere_dict.keys()])
    
    #Isolate bio_params of interest: from corr_ef_code and with lognormal uncertainty
    corr_input_indices=[biosphere_dict[('biosphere3',code)] for code in corr_ef_code_input if code in set_efs]
    corr_output_indices=[biosphere_dict[('biosphere3',code)] for code in corr_ef_code_output if code in set_efs]
    corr_twin_indices=[(biosphere_dict[('biosphere3',code[0])],biosphere_dict[('biosphere3',code[1])]) for code in corr_twin_ef_input_output if (code[0] in set_efs and code[1] in set_efs)]
    corr_input_twin_indices=[indice[0] for indice in corr_twin_indices]
    corr_output_twin_indices=[indice[1] for indice in corr_twin_indices]
    corr_twin_indices_dict={indice[0]:indice[1] for indice in corr_twin_indices}
    
    #print(corr_twin_indices_dict)
    
    set_input=set(corr_input_indices)
    set_output=set(corr_output_indices)
    set_input_output=set(corr_input_indices+corr_output_indices)
    
    set_twin=set(corr_twin_indices)
    set_input_twin=set(corr_input_twin_indices)
    set_output_twin=set(corr_output_twin_indices)
    set_input_output_twin=set(corr_input_twin_indices+corr_output_twin_indices)
    
    
    selected_bio_params=np.array([bio_param for bio_param in bio_params if (bio_param['row'] in set_input_output and bio_param['uncertainty_type']==2)])
    
    #Per activity
    set_col_indices=list(set(selected_bio_params['col']))
    
    for col in set_col_indices:
        
        act_bio_params=np.array([bio_param for bio_param in selected_bio_params if bio_param['col']==col])
        
        #Test if {input}=0 or {output}=0 --> exclude the activity from the list
        act_input_params=np.array([bio_param for bio_param in act_bio_params if bio_param['row'] in set_input])
        act_output_params=np.array([bio_param for bio_param in act_bio_params if bio_param['row'] in set_output])
        
        try:
            act_input_params[0]
        except IndexError:
            continue
            
        try:
            act_output_params[0]
        except IndexError:
            continue
            
        
        #Test if input=output for twin ef --> remove uncertainty from the bio_params
        #act_input_twin_params=np.array([bio_param for bio_param in act_input_params if (bio_param['row'] in set_input_twin and bio_param['row'] in set_input_output_twin)])
        #act_output_twin_params=np.array([bio_param for bio_param in act_output_params if (bio_param['row'] in set_output_twin and bio_param['row'] in set_input_output_twin)])
        #
        #try:
        #    act_input_twin_params[0]
        #    
        #    for twin_input_row in act_input_twin_params['row']:
        #        twin_output_row=corr_twin_indices_dict[twin_input_row]
        #        
        #        output_twin_params=np.array([bio_param for bio_param in act_output_twin_params if bio_param['row']==twin_output_row])
        #        
        #        try:
        #            output_twin_param=output_twin_params[0]
        #            input_twin_param=np.array([bio_param for bio_param in act_input_twin_params if bio_param['row']==twin_input_row])[0]
        #            
        #            input_row_col_to_remove=(input_twin_param['row'],input_twin_param['col'])
        #            output_row_col_to_remove=(output_twin_param['row'],output_twin_param['col'])
        #                                
        #            if input_twin_param['scale']==output_twin_param['scale']:                        
        #                input_condition=np.where((corr_bio_params['row'] == input_row_col_to_remove[0]) * (corr_bio_params['col'] == input_row_col_to_remove[1]))
        #                corr_bio_params[input_condition[0].tolist()[0]]['uncertainty_type']=1
        #                act_input_params=[bio_param for bio_param in act_input_params if (bio_param['row'] != input_row_col_to_remove[0] and bio_param['col'] != input_row_col_to_remove[1])]
        #                
        #                output_condition=np.where((corr_bio_params['row'] == output_row_col_to_remove[0]) * (corr_bio_params['col'] == output_row_col_to_remove[1]))
        #                corr_bio_params[output_condition[0].tolist()[0]]['uncertainty_type']=1
        #                act_output_params=[bio_param for bio_param in act_output_params if (bio_param['row'] != output_row_col_to_remove[0] and bio_param['col'] != output_row_col_to_remove[1])]
        #                
        #                print(col,biosphere_names[inv_biosphere_dict[input_row_col_to_remove[0]]],biosphere_names[inv_biosphere_dict[output_row_col_to_remove[0]]])
        #                
        #        except IndexError:
        #            pass
        #    
        #except IndexError:
        #    pass
            
        
        #Calculate Aij ratio lognormal rv and Store Aij and associate it with Yj only
        for input_param in act_input_params:
            for output_param in act_output_params:
                gmean_Xi=input_param['loc']
                gmean_Yj=output_param['loc']
                gsigma_Xi=input_param['scale']
                gsigma_Yj=output_param['scale']
                params_Aij_dict=calculate_ratio_corr_lognormal(gmean_Xi,gmean_Yj,gsigma_Xi,gsigma_Yj)
                ratio_rv_dict_per_activity[col]={output_param['row']:{input_param['row']:params_Aij_dict}}
                
                #print(type(params_Aij_dict['scale']))
                
                if np.isnan(params_Aij_dict['scale']):
                    print(col,biosphere_names[inv_biosphere_dict[input_param['row']]],biosphere_names[inv_biosphere_dict[output_param['row']]])
                                                 
    return ratio_rv_dict_per_activity; #return (ratio_rv_dict_per_activity,corr_bio_params);

#Each iteration after regenerating the B matrix
def generate_corr_biosphere_matrix(ratio_rv_dict_per_activity,biosphere_dict, biosphere_matrix):
    
    corr_biosphere_matrix=biosphere_matrix.copy()
    
    #per activity
    for col,A in ratio_rv_dict_per_activity.items():
        
        #per output Yj
        for output_row,Aj in A.items():
            
            #print(Aj)

            #Retrieve sum of estimation of Xi
            input_rows=Aj.keys()
            sum_xi=sum([biosphere_matrix[row,col] for row in input_rows])

            #Generate Aij
            Aij_rvs = stats_arrays.UncertaintyBase.from_dicts(*Aj.values())
            Aij_rng = stats_arrays.MCRandomNumberGenerator(Aij_rvs)
            aij=next(Aij_rng)
            sum_aij=sum(aij)
                
            #Calculate Yj
            yj=sum_xi/sum_aij

            #Replace Yj values in the biosphere_matrix
            corr_biosphere_matrix[output_row,col]=yj
    
    return corr_biosphere_matrix;

In [43]:
act=DB_eiv33.get('f868a8bb1c627e75df31949ecdc347b9')
LCA_obj=bw.LCA({act:1})
LCA_obj.lci()

In [44]:
land_transfo_efs=[ef for ef in biosphere3 if 'Transformation, ' in str(ef)]
land_transfo_input={ef['name'].rsplit('Transformation, from ',1)[1]:ef for ef in land_transfo_efs if ', from' in str(ef)}
land_transfo_output={ef['name'].rsplit('Transformation, to ',1)[1]:ef for ef in land_transfo_efs if ', to' in str(ef)}
corr_twin_land_transfo=[(land_transfo_input[end_name]['code'],land_transfo_output[end_name]['code']) for end_name in land_transfo_input.keys()]
land_transfo_code_input=[ef['code'] for ef in land_transfo_input.values()]
land_transfo_code_output=[ef['code'] for ef in land_transfo_output.values()]


len(land_transfo_efs), len (land_transfo_input), len(land_transfo_output),len(corr_twin_land_transfo)

(120, 60, 60, 60)

In [45]:
corr_ef_code_input=land_transfo_code_input
corr_ef_code_output=land_transfo_code_output
corr_twin_ef_input_output=corr_twin_land_transfo
biosphere_dict=LCA_obj.biosphere_dict
bio_params=LCA_obj.bio_params
biosphere_names={('biosphere3',ef['code']):ef['name'] for ef in biosphere3}

ratio_rv_dict_per_activity=calculate_ratio_corr_lognormal_for_DB(corr_ef_code_input, 
                              corr_ef_code_output, 
                              corr_twin_ef_input_output, 
                              biosphere_dict, 
                              bio_params,biosphere_names)

In [46]:
ratio_rv_dict_per_activity

{8193: {1655: {635: {'loc': 0.0,
    'scale': 0.00011423815239643584,
    'uncertainty_type': 2}}},
 2: {1655: {635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}},
 1: {1549: {697: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}},
 8201: {1655: {831: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}},
 8204: {1655: {635: {'loc': 0.0,
    'scale': 0.00010805032449841224,
    'uncertainty_type': 2}}},
 8206: {1655: {635: {'loc': 0.0,
    'scale': 5.4650991414893778e-05,
    'uncertainty_type': 2}}},
 8207: {1655: {635: {'loc': 0.0,
    'scale': 1.067988297751739e-05,
    'uncertainty_type': 2}}},
 8219: {1655: {635: {'loc': 0.0,
    'scale': 0.00011997978239926968,
    'uncertainty_type': 2}}},
 31: {1655: {635: {'loc': 4.5886412, 'scale': 1e-13, 'uncertainty_type': 2}}},
 32: {1655: {635: {'loc': 4.5886412, 'scale': 1e-13, 'uncertainty_type': 2}}},
 33: {211: {1060: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}},
 36: {1865: {622: {'loc': -1.2378744, 'scale': 1e-1

In [47]:
biosphere_matrix=LCA_obj.biosphere_matrix
corr_biosphere_matrix=generate_corr_biosphere_matrix(ratio_rv_dict_per_activity,biosphere_dict, biosphere_matrix)

{635: {'loc': 0.0, 'scale': 0.00011423815239643584, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{697: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{831: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 0.00010805032449841224, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 5.4650991414893778e-05, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1.067988297751739e-05, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 0.00011997978239926968, 'uncertainty_type': 2}}
{635: {'loc': 4.5886412, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 4.5886412, 'scale': 1e-13, 'uncertainty_type': 2}}
{1060: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{622: {'loc': -1.2378744, 'scale': 1e-13, 'uncertainty_type': 2}}
{1828: {'loc': 3.9102449, 'scale': 6.4041924814935562e-05, 'uncertainty_type': 2}}
{1338: {'loc': 0.0, 'scale': 0.48501355369637095, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 5.5

{825: {'loc': -4.0283828, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 4.6051702, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{622: {'loc': -1.2378678, 'scale': 1e-13, 'uncertainty_type': 2}}
{1337: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 2.8899341369872459e-05, 'uncertainty_type': 2}}
{468: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{622: {'loc': 0.0, 'scale': 3.782811140342793e-05, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 4.6051702, 'scale': 0.004996878777998019, 'uncertainty_type': 2}}
{635: {'loc': 0.0009431839, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1.067988297751739e-05, 'uncertainty_type': 2}}
{64: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{697: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.

{49: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{831: {'loc': 0.0, 'scale': 2.8322682199320788e-05, 'uncertainty_type': 2}}
{1060: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 4.6051702, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.55961561, 'scale': 1e-13, 'uncertainty_type': 2}}
{1828: {'loc': 3.9102449, 'scale': 2.8575470044993949e-05, 'uncertainty_type': 2}}
{1741: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{1933: {'loc': 0.0, 'scale': 0.00015506193875145679, 'uncertainty_type': 2}}
{622: {'loc': -1.2378744, 'scale': 1e-13, 'uncertainty_type': 2}}
{1741: {'loc': -0.34249032, 'scale': 1e-13, 'uncertainty_type': 2}}
{229: {'loc': 0.0, 'scale': 6.4703429478738333e-05, 'uncertainty_type': 2}}
{1338: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{831: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 4.6051702, 'scale': 0.1519835226189544, 'uncertainty_

{1933: {'loc': 0.0, 'scale': 0.00015506193875145679, 'uncertainty_type': 2}}
{49: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{622: {'loc': -1.2378744, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.00063705444, 'scale': 0.00012154394080465316, 'uncertainty_type': 2}}
{229: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{468: {'loc': -0.81580782, 'scale': 3.5879255068738936e-06, 'uncertainty_type': 2}}
{468: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1.067988297751739e-05, 'uncertainty_type': 2}}
{1338: {'loc': -0.56945515, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{229: {'loc': 0.0, 'scale': 1e-13, 'uncertainty_type': 2}}
{1338: {'loc': 0.69314671, 'scale': 1e-13, 'uncertainty_type': 2}}
{635: {'loc': 0.0, 'scale': 0.00010805032449841224, 'uncertainty_

In [48]:
corr_biosphere_matrix != biosphere_matrix

<1960x13831 sparse matrix of type '<class 'numpy.bool_'>'
	with 1456 stored elements in Compressed Sparse Row format>

In [49]:
len(ratio_rv_dict_per_activity)

1456